In [1]:
# Import packages
import os
import tweepy as tw
import pandas as pd
import numpy as np
import re 

folder_name = 'filtered_data/'
# Create data folder
if not os.path.isdir('./'+folder_name):
    os.mkdir('./'+folder_name)

## Manipulate data and create a complete dataframe

In [ ]:
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
year_list = ['17','18','19','20']

dft_list = []
dfr_list = []

for year in year_list:
    for month in month_list:
        path = 'collected_data/data_'+year+'/20'+year+'-'+month+'_tweet.xlsx'
        dft = pd.read_excel(path,engine='openpyxl',index_col=0,header=[0,1])
        del dft[('tweet','in_reply_to_screen_name')]
        dft_list.append(dft)
        
        path = 'collected_data/data_'+year+'/20'+year+'-'+month+'_retweet.xlsx'
        dfr = pd.read_excel(path,engine='openpyxl',index_col=0,header=[0,1])
        del dfr[('tweet','id_original')]
        dfr_list.append(dfr)

    df_tweet = pd.concat(dft_list,ignore_index=True)
    df_retweet = pd.concat(dfr_list,ignore_index=True)
    print('Dimension of 20'+year+' tweet dataset  :',df_tweet.shape)
    print('Dimension of 20'+year+' retweet dataset:',df_retweet.shape,'\n') 
     
    dft_list = []
    dfr_list = []

    path = 'collected_data/data_all/20'+year+'-all_tweet.xlsx'
    df_tweet_all = pd.read_excel(path,engine='openpyxl',index_col=0,header=[0,1])
    del df_tweet_all[('tweet','in_reply_to_screen_name')]
        
    path = 'collected_data/data_all/20'+year+'-all_retweet.xlsx'
    df_retweet_all = pd.read_excel(path,engine='openpyxl',index_col=0,header=[0,1])
    del df_retweet_all[('tweet','id_original')]

    print('Dimension of 20'+year+' tweet all dataset  :',df_tweet_all.shape)
    print('Dimension of 20'+year+' retweet all dataset:',df_retweet_all.shape,'\n') 
    
    data = pd.concat([df_tweet,df_retweet,df_tweet_all,df_retweet_all],ignore_index=True)
    print('\nDimension of 20'+year+' entire dataset:',data.shape)
    
    # Remove city None
    data = data[~data[("user","location")].isin(['none'])]

    # Remove duplicated tweet id
    data = data.drop_duplicates(subset=[('tweet','id')])
    print('\nDimension of 20'+year+' dataset: (after removing duplicates tweet id and None):',data.shape)    
    
    # Add tag column
    data[('user','tag')] = 1
    
    print('\n\n\n')
    # Convert to a file excel
    data.to_excel(folder_name+'dataset_20'+year+'.xlsx') 

## Manipulate dataframe

In [7]:
df_2017 = pd.read_excel(folder_name+'tag_dataset_2017.xlsx',engine='openpyxl',index_col=0,header=[0,1])
df_2018 = pd.read_excel(folder_name+'tag_dataset_2018.xlsx',engine='openpyxl',index_col=0,header=[0,1])
df_2019 = pd.read_excel(folder_name+'tag_dataset_2019.xlsx',engine='openpyxl',index_col=0,header=[0,1])
df_2020 = pd.read_excel(folder_name+'tag_dataset_2020.xlsx',engine='openpyxl',index_col=0,header=[0,1])

data = pd.concat([df_2020],ignore_index=True)
#data = pd.concat([df_2020],ignore_index=True)

print('Dataset dim before tag:', data.shape[0])

# Filter only for novax people
data = data[data[('user','tag')]==1]

# Fill lat and lon province for user with only the region information by associating capitals
df_reg = pd.read_excel('../useful_datasets/italian_regions_new.xlsx',engine='openpyxl')

regions = df_reg['region']
province = df_reg['capitals']
lat = df_reg['lat']
lon = df_reg['lon']




# TO FIX BOLZANO AND TRENTO PROBLEM!!!!!!!!!!!!!
cit = ['bolzano','trento']
for i in range(len(cit)):
    mask = data[("user","province")].notnull() & data[("user","province")].str.contains(fr'\b{cit[i]}\b', regex=True, case=False)
    data.loc[mask,("user","region")] = cit[i]

# TO CHANGE IF WE WANT TO DO PROVINCE ANALYSIS!!!!!!!!!!!!!!!!!!
for i in range(len(regions)):
    # If city is null, associate latitude and longitude of the capital of the region
    mask = data[("user","region")].str.contains(fr'\b{regions[i]}\b', regex=True, case=False)
    data.loc[mask,("user","province")] = province[i] 
    data.loc[mask,("user","lat_province")] = lat[i] 
    data.loc[mask,("user","lon_province")] = lon[i]


    
#for i in range(len(regions)):
#    # If city is null, associate latitude and longitude of the capital of the region
#    mask = data[("user","city")].isnull() & data[("user","region")].str.contains(fr'\b{regions[i]}\b', regex=True, case=False)
#    data.loc[mask,("user","province")] = province[i] 
#    data.loc[mask,("user","lat_province")] = lat[i] 
#    data.loc[mask,("user","lon_province")] = lon[i]
    

# Add column total population per province
df_pop = pd.read_excel('../useful_datasets/abitanti_per_regione.xlsx',engine='openpyxl')

region = df_pop['region']
tweet_activity = df_pop['tweet_percentage']

for i in range(len(region)):
    mask = data[("user","region")].str.contains(fr'\b{region[i]}\b', regex=True, case=False)
    data.loc[mask,("user","tweet_activity")] = tweet_activity[i] 

data["tweet","if_retweet"] = 0

for i in range(len(region)):
    mask = data[("tweet","text")].str.contains('RT @', regex=True, case=True)
    data.loc[mask,("tweet","if_retweet")] = 1
    
# Add column total population per province
#df_pop = pd.read_excel('../useful_datasets/pop_province.xlsx',engine='openpyxl')

#province = df_pop['Province']
#pop_total = df_pop['Total']

#for i in range(len(province)):
#    mask = data[("user","province")].str.contains(fr'\b{province[i]}\b', regex=True, case=False)
#    data.loc[mask,("user","total_population")] = pop_total[i] 
print('Dataset dim after tag:', data.shape[0])

Dataset dim before tag: 277
Dataset dim after tag: 228


In [3]:
# Associate a weight to each tweet of a user
#data[('user','user_weight')] = (1 + data[('tweet','favorites_count')] + 5*data[('tweet','retweet_count')])/data[('user','total_population')]
mask = data[("tweet","if_retweet")]==0
data.loc[mask,("user","user_weight_p")] = (1 + data[('tweet','favorites_count')] + 5*data[('tweet','retweet_count')])/data[('user','tweet_activity')]
  
mask = data[("tweet","if_retweet")]==1 
data.loc[mask,("user","user_weight_p")] = (1 + data[('tweet','favorites_count')])/data[('user','tweet_activity')]
 
#data[('user','user_weight_p')] = (1 + data[('tweet','favorites_count')] + 5*data[('tweet','retweet_count')])/data[('user','tweet_activity')]

# Associate to each unique ID the sum of the corresponding weights
data[('user','user_weight')] = data[('user','id')].map( data['user'].groupby(('id'))['user_weight_p'].agg('sum') )

data_unique_user = data.drop_duplicates(subset=[('user','id')])

# Associate to each unique province the sum of the corresponding weights of all the utents
#data[('user','province_weight')] = data[('user','province')].map( data['user'].groupby(('province'))['user_weight'].agg('sum') )
data_unique_user[('user','region_weight')] = data_unique_user[('user','region')].map( data_unique_user['user'].groupby(('region'))['user_weight'].agg('sum') )

# Associate to each unique province the sum of the corresponding weights of all the utents
####data[('user','region_weight')] = data[('user','region')].map( data['user'].groupby(('region'))['user_weight'].agg('sum') )

data_unique_user

/Users/alicepagano/anaconda3/envs/life/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


user                      \
         screen_name                  id   
1        wushurabbit           489909029   
3            MIMIGIO           503303414   
4    vecchiaquercia8  824996743276331008   
5      Yle1807081486  824603877450321920   
6       robertaestop           171040838   
..               ...                 ...   
418       stefi1966r          2401743697   
419    GiuliaOhana92  916545109864349696   
421       ZiuPaddori           586715112   
426       XMERIDIO78  766601799155449856   
436   GastoneMappini  773240110120927232   

                                                        \
                                           description   
1    Idee, fatti e ricerche controverse che potrebb...   
3                            PACE GIUSTIZIA E LIBERTA'   
4                                                 None   
5    Se vivi la vita in punta di piedi d accordo no...   
6                      Lottare...combattere...GO ALF ♥   
..                                                 ...   
418  Il bello della vita è esserci 😍. I cafoni li b...   
419  25 anni, mamma, doula, blogger, reiki, meditaz...   
421  living on bf43 world with a look on real world...   
426  L'invidia è una confessione d'inferiorità.\nMa...   
436  Non ho colore politico ♐️ Non nobis Domine, no...   

                                                              \
                   location followers_count favourites_count   
1                      roma            1695            72013   
3                    napoli            1178            82323   
4    ferrara emilia romagna              15              100   
5           livorno toscana              19              121   
6                   liguria             391              370   
..                      ...             ...              ...   
418              roma lazio             829             3996   
419         ranco lombardia             104               14   
421        milano lombardia             119             2798   
426           puglia italia            4285            81089   
436         firenze toscana            1145            20760   

                                             tweet                             \
                    created_at                  id                 created_at   
1   2012-02-12 01:29:04.000000  826350384901939200 2017-01-31 08:44:19.000000   
3   2012-02-25 17:30:46.000000  825005042902134784 2017-01-27 15:38:23.999999   
4   2017-01-27 15:05:26.000000  824997962115903488 2017-01-27 15:10:16.000000   
5   2017-01-26 13:04:19.000000  824951154513678336 2017-01-27 12:04:15.999999   
6   2010-07-26 12:10:04.000000  824926393389740032 2017-01-27 10:25:52.999999   
..                         ...                 ...                        ...   
418 2014-03-21 15:28:03.000001  924749186750443520 2017-10-29 21:26:01.000000   
419 2017-10-07 06:05:57.000000  916763739914989568 2017-10-07 20:34:42.000000   
421 2012-05-21 15:31:43.000000  908442923032772608 2017-09-14 21:30:45.000000   
426 2016-08-19 11:44:46.000000  905765199222857728 2017-09-07 12:10:26.000000   
436 2016-09-06 19:23:03.000000  900842180327202816 2017-08-24 22:08:07.000000   

                                                        ...     user           \
                                                  text  ...     city province   
1    Vaccini tra scienza e diritto.\n\nhttps://t.co...  ...     roma     roma   
3    RT @Laurastixi: Fate una bella inchiesta sui m...  ...   napoli   napoli   
4                                  #noobbligovaccinale  ...  ferrara  bologna   
5    RT @chierighini66: #noobbligovaccinale\nPrima ...  ...  livorno  firenze   
6    #Lorenzin #NoVaccini #LibertaDiScelta #Vaccini...  ...      NaN   genova   
..                                                 ...  ...      ...      ...   
418  RT @tiecolino: 40mila secondo la questura \n4 ...  ...     roma     roma   
419  RT @tiecolino: 40mila secondo la questura \n4 ...  ...    ranco   m

In [17]:
data_unique_region = data_unique_user.drop_duplicates(subset=[('user','region')])

# Convert to a file excel
data_unique_region.to_excel(folder_name+'tweet_region_dataset_20.xlsx') 
#data_unique_region.to_excel(folder_name+'tweet_region_dataset_20.xlsx') 
print('Dimension of the entire dataset (after removing duplicates regions):',data_unique_region.shape) 

Dimension of the entire dataset (after removing duplicates regions): (20, 26)


In [18]:
# Simple check
#data[data[('user','province')]=="reggio nell'emilia"].drop_duplicates(subset=[('user','id')])

In [19]:
#data_unique_province = data.drop_duplicates(subset=[('user','province')])
# Convert to a file excel
#data_unique_province.to_excel(folder_name+'tweet_province_dataset_17-18-19.xlsx') 
#print('Dimension of the entire dataset (after removing duplicates provinces):',data_unique_province.shape) 